# 04: Bounded Contexts — Multi-Instance Kind Definitions

The `examples/shop/` fixture contains a clean-architecture e-commerce shop with domain entities, application ports/use-cases, and infrastructure adapters.

This notebook:
1. Manually writes Kind definitions, an `Instance<T>` instance, and type-level constraints for the shop codebase
2. Restructures into two bounded contexts (payments + orders) sharing the same Kind type
3. Demonstrates that contracts are enforced across all instances

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

## Setup

In [ ]:
import { PROJECT_ROOT, KSC, ksc, tree, copyDir, copyFixture, writeFile, readFile, cleanup } from './lib.ts';

console.log("CLI path:", KSC);

---

## Part 1: Setting Up KindScript on the Example Codebase

We start with the shop fixture's source files. We'll write the Kind definitions, instance mapping, and contracts by hand.

In [ ]:
const DEMO = copyFixture("shop");

console.log("Working directory:", DEMO);
console.log("\n=== Project structure (no KindScript config yet) ===");
await tree(DEMO);

In [ ]:
// ─── Write src/context.ts by hand ───

const singleArchitecture = `
import type { Kind, Constraints, Instance } from 'kindscript';

// ─── Kind Definitions ───
// Three leaf layers, and a composite context that groups them.

export type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;
export type ApplicationLayer = Kind<"ApplicationLayer">;
export type InfrastructureLayer = Kind<"InfrastructureLayer">;

export type ShopContext = Kind<"ShopContext", {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [
    ["domain", "application"],
    ["domain", "infrastructure"],
    ["application", "infrastructure"],
  ];
  mustImplement: [["application", "infrastructure"]];
}>;

// ─── Instance ───
// Maps the Kind type to real directories on disk.
// Root is inferred from this file's directory (src/).

export const shop = {
  domain: {},
  application: {},
  infrastructure: {},
} satisfies Instance<ShopContext>;
`.trimStart();

writeFile(DEMO, "src/context.ts", singleArchitecture);

console.log("=== src/context.ts ===");
console.log(singleArchitecture);

**`src/context.ts`** imports types from the `kindscript` package (`import type` — zero runtime footprint), then defines:
1. **Kind definitions** — type aliases using `Kind<N>` and `Kind<N, Members, Constraints>`. `ShopContext` is a composite that groups `DomainLayer`, `ApplicationLayer`, and `InfrastructureLayer` as members. Constraints (like `noDependency`) are declared as the third type parameter.
2. **Instance** — `{ ... } satisfies Instance<ShopContext>` maps the Kind type to real directories. The root is inferred from the definition file's directory (`src/`), so member keys give `src/domain`, `src/application`, `src/infrastructure`.
3. **Intrinsic constraints** — `DomainLayer` has `{ pure: true }` which automatically generates purity enforcement when used as a member.

Definition files are auto-discovered — no `kindscript.json` needed. The Kind type is the single source of truth for all architectural rules.

In [ ]:
// Verify the generated architecture passes
await ksc("check", DEMO);

All contracts pass on the single-instance setup:
- **noDependency** — domain has no imports from application or infrastructure; application has no imports from infrastructure
- **purity** — domain has no Node.js built-in imports (propagated from `DomainLayer`'s `{ pure: true }`)
- **mustImplement** — every port interface in `src/application/` has an implementing class in `src/infrastructure/`

Now let's scale to multiple bounded contexts.

---

## Part 2: Restructuring into Two Bounded Contexts

We want two bounded contexts — **payments** and **orders** — each with its own domain/application/infrastructure layers, sharing the same Kind definition.

We'll restructure the project: move the current `src/` content into `src/orders/`, and create a new `src/payments/` context.

In [ ]:
// Move existing src/ into src/orders/
const tmpSrc = `${DEMO}/_src_backup`;
Deno.renameSync(`${DEMO}/src`, tmpSrc);
Deno.mkdirSync(`${DEMO}/src/orders`, { recursive: true });
copyDir(`${tmpSrc}/domain`, `${DEMO}/src/orders/domain`);
copyDir(`${tmpSrc}/application`, `${DEMO}/src/orders/application`);
// Only copy the adapters (not api.ts entry point)
Deno.mkdirSync(`${DEMO}/src/orders/infrastructure`, { recursive: true });
Deno.copyFileSync(`${tmpSrc}/infrastructure/sql-order-repository.ts`, `${DEMO}/src/orders/infrastructure/sql-order-repository.ts`);
Deno.copyFileSync(`${tmpSrc}/infrastructure/in-memory-catalog.ts`, `${DEMO}/src/orders/infrastructure/in-memory-catalog.ts`);
Deno.copyFileSync(`${tmpSrc}/infrastructure/email-notification.ts`, `${DEMO}/src/orders/infrastructure/email-notification.ts`);

// Create src/payments/ with its own domain (copied) + payment-specific application/infra
copyDir(`${tmpSrc}/domain`, `${DEMO}/src/payments/domain`);

writeFile(DEMO, "src/payments/application/payment-gateway.port.ts", `
export interface PaymentGatewayPort {
  charge(customerId: string, amountCents: number): boolean;
  refund(transactionId: string): boolean;
}
`);

writeFile(DEMO, "src/payments/application/process-payment.ts", `
import { Money } from '../domain/money';
import { PaymentGatewayPort } from './payment-gateway.port';

export function processPayment(
  customerId: string,
  amount: Money,
  gateway: PaymentGatewayPort,
): boolean {
  return gateway.charge(customerId, Math.round(amount.amount * 100));
}
`);

writeFile(DEMO, "src/payments/infrastructure/stripe-gateway.ts", `
import { PaymentGatewayPort } from '../application/payment-gateway.port';

export class StripeGateway implements PaymentGatewayPort {
  charge(customerId: string, amountCents: number): boolean {
    console.log('[Stripe] Charging', customerId, amountCents, 'cents');
    return true;
  }
  refund(transactionId: string): boolean {
    console.log('[Stripe] Refunding', transactionId);
    return true;
  }
}
`);

Deno.removeSync(tmpSrc, { recursive: true });

console.log("=== Restructured project ===");
await tree(DEMO);

### Rewrite definitions for Multi-Instance

The single-instance architecture maps one `ShopContext` to `src/`. Now we need a shared `BoundedContext` Kind with two definition files — one in each bounded context directory. Each file's directory becomes the root for that context.

In [ ]:
// Remove the old single-instance definition
try { Deno.removeSync(`${DEMO}/src/context.ts`); } catch {}

const sharedKindDefs = `
import type { Kind, Constraints, Instance } from 'kindscript';

// ─── Kind Definitions ───

export type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;
export type ApplicationLayer = Kind<"ApplicationLayer">;
export type InfrastructureLayer = Kind<"InfrastructureLayer">;

export type BoundedContext = Kind<"BoundedContext", {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [
    ["domain", "application"],
    ["domain", "infrastructure"],
    ["application", "infrastructure"],
  ];
  mustImplement: [["application", "infrastructure"]];
}>;
`.trimStart();

// ─── Two definition files, one per bounded context ───

const paymentsKts = sharedKindDefs + `
// Root is inferred from this file's directory: src/payments/
export const payments = {
  domain: {},
  application: {},
  infrastructure: {},
} satisfies Instance<BoundedContext>;
`;

const ordersKts = sharedKindDefs + `
// Root is inferred from this file's directory: src/orders/
export const orders = {
  domain: {},
  application: {},
  infrastructure: {},
} satisfies Instance<BoundedContext>;
`;

writeFile(DEMO, "src/payments/payments.ts", paymentsKts);
writeFile(DEMO, "src/orders/orders.ts", ordersKts);

console.log("=== src/payments/payments.ts ===");
console.log(paymentsKts);
console.log("\n=== src/orders/orders.ts ===");
console.log(ordersKts);

---

## Step 1: Check Contracts — Clean State

Both bounded contexts should satisfy all contracts.

In [ ]:
await ksc("check", DEMO);

All contracts pass on the single-instance setup:
- **noDependency** — domain has no imports from application or infrastructure; application has no imports from infrastructure
- **purity** — domain has no Node.js built-in imports (propagated from `DomainLayer`'s `{ pure: true }`)
- **mustImplement** — every port interface in `src/application/` has an implementing class in `src/infrastructure/`

Now let's scale to multiple bounded contexts.

---

## Step 2: Introduce a Violation in Payments

The payments domain has clean `Money`, `Order`, `Product`, and `Customer` entities copied from the example codebase. Let's break purity by adding a Node.js import to `money.ts` — the violation should be caught even though payments isn't the last-declared instance.

In [ ]:
// Save the clean file so we can restore it later
const cleanMoney = readFile(DEMO, "src/payments/domain/money.ts");

// Inject a forbidden import at the top
writeFile(DEMO, "src/payments/domain/money.ts",
  `import * as crypto from 'crypto';\n\n` + cleanMoney
);

console.log("=== payments/domain/money.ts (first 5 lines) ===");
console.log(readFile(DEMO, "src/payments/domain/money.ts").split('\n').slice(0, 5).join('\n'));
console.log("...");
console.log("\nViolation: domain imports 'crypto' (impure)");

In [ ]:
await ksc("check", DEMO);

**The violation is caught.** KindScript reports `KS70003` for the impure `crypto` import in `payments/domain/money.ts`.

Both bounded contexts share the same `BoundedContext` Kind type, and the classifier generates contracts for *all* instances — not just the last one. The `payments` context is correctly enforced.

---

## Step 3: Verify — Orders Violations Are Also Caught

Let's restore payments and break the orders context with a different violation type (noDependency).

In [ ]:
// Restore clean payments domain
writeFile(DEMO, "src/payments/domain/money.ts", cleanMoney);

// Break orders domain: import from infrastructure (noDependency violation)
const cleanOrder = readFile(DEMO, "src/orders/domain/order.ts");

writeFile(DEMO, "src/orders/domain/order.ts",
  `import { SqlOrderRepository } from '../infrastructure/sql-order-repository';\n\n` + cleanOrder
);

console.log("Payments domain: restored (clean)");
console.log("Orders domain:   noDependency violation introduced (imports from infrastructure)");

In [ ]:
await ksc("check", DEMO);

The violation is caught — `KS70001` for the forbidden dependency in `orders/domain/order.ts`. Both contexts are fully enforced regardless of declaration order.

In [ ]:
// Restore clean state
writeFile(DEMO, "src/orders/domain/order.ts", cleanOrder);
await ksc("check", DEMO);

---

## Step 4: Existence Checking — Adding a Third Context

Existence checking (`filesystem.exists`) is opt-in. The payments and orders contexts don't use it, so missing directories are silently ignored. Let's add a `shipping` context with a stricter Kind that includes `filesystem.exists`.

In [ ]:
// Add a third context with filesystem.exists constraint — but no directories yet
const shippingKts = `
import type { Kind, Constraints, Instance } from 'kindscript';

// Same leaf layers as before
export type DomainLayer = Kind<"DomainLayer", {}, { pure: true }>;
export type ApplicationLayer = Kind<"ApplicationLayer">;
export type InfrastructureLayer = Kind<"InfrastructureLayer">;

// Add filesystem.exists to enforce directory existence
export type StrictBoundedContext = Kind<"StrictBoundedContext", {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [
    ["domain", "application"],
    ["domain", "infrastructure"],
    ["application", "infrastructure"],
  ];
  mustImplement: [["application", "infrastructure"]];
  filesystem: {
    exists: ["domain", "application", "infrastructure"];
  };
}>;

// Root is inferred from this file's directory: src/shipping/
export const shipping = {
  domain: {},
  application: {},
  infrastructure: {},
} satisfies Instance<StrictBoundedContext>;
`;

writeFile(DEMO, "src/shipping/shipping.ts", shippingKts);
await ksc("check", DEMO);

`KS70010` fires for all three missing directories under `src/shipping/`. The `filesystem: { exists: [...] }` constraint makes existence checking opt-in — only members listed in the array are checked. Without this constraint, missing directories are silently ignored (dependency contracts simply pass when a directory has no files).

We can fix this by creating the directories.

In [ ]:
// Create the missing directories
Deno.mkdirSync(`${DEMO}/src/shipping/domain`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/shipping/application`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/shipping/infrastructure`, { recursive: true });

console.log("--- After creating directories ---");
await ksc("check", DEMO);

---

## Design Discussion

### 1. Shared vs Per-Instance Constraints

With the shared-Kind approach, constraints on the Kind type apply to **all** instances. This is the intended design — the Kind type is the single source of truth for all architectural rules.

If you need **different rules per context** (e.g., payments has stricter purity), you must use separate Kind types. This is a fundamental design choice: same Kind type = same constraints, different constraints = different Kind type.

### 2. Cross-Context Dependencies

KindScript's `noDependency` contracts operate *within* a bounded context (e.g., payments.domain cannot import from payments.infrastructure). But what about **cross-context** dependencies (payments importing from orders)?

This isn't currently expressible. You'd need a higher-level Kind that has the bounded contexts as members and defines inter-context dependency rules.

In [ ]:
cleanup(DEMO);
console.log("Working directory cleaned up.");

---

## Summary

| Step | What we did |
|------|-------------|
| Write `src/context.ts` | Defined Kind types with constraints, an `Instance` instance |
| `ksc check` | Verified all contracts pass on the single-instance setup |
| Restructure | Moved source files into `src/payments/` and `src/orders/` bounded contexts |
| Write definition file per context | Shared `BoundedContext` Kind with separate definition files per directory |
| Violations | Verified purity and noDependency violations are caught in both contexts |
| Existence | Verified existence checking works across all instances |

| Pattern | Status |
|---------|--------|
| Single instance | Works — all contracts enforced |
| Multi-instance, shared Kind type | Works — all instances get contracts |
| Existence checking | Works — across all instances |

**Key takeaway:** The shared-Kind multi-instance pattern is the recommended approach for bounded contexts. Define the Kind type once, then create a definition file in each context directory with `satisfies Instance<T>`. All instances automatically inherit the Kind's constraints.